In [6]:
import json
import csv
import tweepy
import re
import pandas as pd

In [7]:
## Twitter API credentials
consumer_key = "xgBXi2k3DrVFiW9B2O1qzktO1"
consumer_secret = "gdE616eb3l0i1ahtuZM87T31qrTiUOjyLgwW0k5MBBZOZQXqVA"
access_key = "1453462826060763142-Kn3InEkvIIakUxu4dkoyXlw8J3mtOq"
access_secret = "IS2TL9vYcFkLh29hggRO1nQIsfb74cpHuHmsetMG5QpCS"

In [8]:
## Authorize twitter, initialize tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)
api

In [9]:
## Function to get tweets

def get_all_tweets(screen_name):
    #Twitter only allows access to a users most recent 3240 tweets with this method

    #initialize a list to hold all the tweepy Tweets
    alltweets = []
    
    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    
    #save most recent tweets
    alltweets.extend(new_tweets)

    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print("getting tweets before %s" % (oldest))

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)

        #save most recent tweets
        alltweets.extend(new_tweets)
    
        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print("...%s tweets downloaded so far" % (len(alltweets)))
    
    #transform the tweepy tweets into a 2D array that will populate the csv	
    outtweets = [[screen_name, tweet.id_str, tweet.created_at, tweet.text, tweet.retweet_count, tweet.favorite_count] for tweet in alltweets]
    tweets_df=pd.DataFrame(outtweets)
    return tweets_df

    pass

In [10]:
ceo_list = pd.read_csv("ceo.csv")
ceo_list.head()

,Usernames
0,@Benioff
1,@BillGates
2,@Bill_Gross
3,@ChuckRobbins
4,@IndraNooyi


In [ ]:
ceo_tweets = pd.DataFrame(columns=['username', 'id', 'date', 'tweet', 'retweets','likes'])
                                   
for handle in ceo_list["Usernames"]:
    ceo_tweets_temp = get_all_tweets(handle)
    ceo_tweets_temp.columns = ['username', 'id', 'date', 'tweet','retweets','likes']
    
    ceo_tweets = ceo_tweets.append(ceo_tweets_temp, ignore_index=True)

getting tweets before 1521642192061693951
...400 tweets downloaded so far
getting tweets before 1504510261100789764
...599 tweets downloaded so far
getting tweets before 1490566567205216257
...798 tweets downloaded so far
getting tweets before 1483268303992672257
...997 tweets downloaded so far
getting tweets before 1473575548450316290
...1196 tweets downloaded so far
getting tweets before 1466968341524856831
...1395 tweets downloaded so far
getting tweets before 1456839790695882759
...1595 tweets downloaded so far
getting tweets before 1453940822341128193
...1794 tweets downloaded so far
getting tweets before 1446532568350806016
...1993 tweets downloaded so far
getting tweets before 1442208988808966153
...2193 tweets downloaded so far
getting tweets before 1440042679925047299
...2392 tweets downloaded so far
getting tweets before 1435872506921947137
...2592 tweets downloaded so far
getting tweets before 1431462345985773568
...2790 tweets downloaded so far
getting tweets before 1428913

In [ ]:
ceo_tweets.to_csv('ceo_tweets.csv')